# Chameleon Assignment

## Results

![Iperf](results_isolated_net.png)

## Reproducing

- Set up nodes using either node_setup.py if you have access to our project, or by following the notebook set up instructions below.
- If you are using a stock image you need to manually install iperf3 on both nodes.
- On one compute node run user@cnode1> iperf3 -s, on the other run user@cnode2> iperf3 -c $CNODE1_IP_OR_HOSTNAME -J > results.json, then store the file.
- Replace the contents of results_closed_network.json in the artifact with your results and run the plotting cell at the bottom.

## Set up nodes from this notebook.

### Generic project set up

In [1]:
import chi

# project set up
chi.set('project_name', 'CHI-221014')
chi.use_site('CHI@Purdue')

Now using CHI@Purdue:
URL: https://chi.rcac.purdue.edu
Location: None
Support contact: help@chameleoncloud.org


### Isolated Network

It is likely that the network already exists in our project, however if this is not the case then it will be created. If you get the IpAddressGenerationFailureClient error on router creation, this will be because you are limited to 1 router and you probably already a router reserved.

In [ ]:
import chi.network

net_name = "stbl"
subnet_name = "private-subnet-stbl-0"
router_name = "public_router"

# automatically set up the network
try:
    network = chi.network.get_network(net_name)
except RuntimeError:
    network = chi.network.create_network(net_name)

try:
    subnet = chi.network.get_subnet(subnet_name)
except RuntimeError:
    subnet = chi.network.create_subnet(subnet_name, network['id'], cidr="10.25.32.0/24")

try:
    router = chi.network.get_router(router_name)
except RuntimeError:
    # may get the error: "IpAddressGenerationFailureClient: No more IP addresses available on network". Delete your current
    # router to resolve.
    router = chi.network.create_router(router_name, "public")
    chi.network.add_subnet_to_router(router['id'], subnet['id'])

### Leases

We attempt to provision the same hardware used to obtain our data, but some or all of it may not be available. If the leases by name don't work, fall back to leasing any head node. If this fails, as we aren't using any special features of the head node, we use a compute node instead. (Unfortunately Jupyter will sometimes only display the error from when it fails to find the named node and not the rest of the output, but creating the lease will still succeed.)

In [ ]:
import chi.lease

head_node, head_lease = [], None

try:
    chi.lease.add_node_reservation(head_node, resource_properties=["==", "$node_name", "snyder-a001"], count=1)
    head_lease = chi.lease.create_lease("CD_Head_Lease", reservations=head_node)
except:
    print("falling back to a default head node")

if head_lease is None:
    try:
        head_node = []
        chi.lease.add_node_reservation(head_node, node_type='indyscc_head', count=1)
        head_lease = chi.lease.create_lease("CD_Head_Lease", reservations=head_node)
    except:
        print("no head nodes available, using a default compute node instead")

# if this fails, we give up!
if head_lease is None:
    head_node = []
    chi.lease.add_node_reservation(head_node, node_type='indyscc_compute', count=1)
    head_lease = chi.lease.create_lease("CD_Head_Lease", reservations=head_node)

# wait for the reservation to start
head_ready = chi.lease.wait_for_active(head_lease['id'])
print("head node lease successful")

As with the head node, if we can't get one of the copmute nodes by name then any compute nodes will have to do.

In [ ]:
import chi.lease

compute_nodes, compute_lease = [], None

try:
    chi.lease.add_node_reservation(compute_nodes, count=1, resource_properties=["==", "$node_name", "rice-a470"])
    chi.lease.add_node_reservation(compute_nodes, count=1, resource_properties=["==", "$node_name", "rice-a330"])
    compute_lease = chi.lease.create_lease("CD_Compute_Lease", reservations=compute_nodes)
except:
    print("falling back to default compute nodes")

if compute_lease is None:
    compute_nodes = []
    chi.lease.add_node_reservation(compute_nodes, count=2, node_type='indyscc_compute')
    compute_lease = chi.lease.create_lease("CD_Compute_Lease", reservations=compute_nodes)

# wait for the reservation to start
compute_ready = chi.lease.wait_for_active(compute_lease['id'])
print("compute node lease successful")

### Servers

If you are running outside the ClusDur project you will not have our image and you may or may not have a keypair. If you do not have the 'indyscc_head-node' image change the image in this file to a stock image. You must also set the key to a previously generated keypair, or generate a new keypair via the GUI.

In [ ]:
import chi.server, chi.lease

# Project-specific default image. Replace with stock image if not run in our project.
head_image = 'indyscc_head-node'
compute_image = 'indyscc-compute-node'
ssh_key = 'CHANGE_ME' # replace with your generated keypair

head = chi.server.create_server(
    "CD_Head",
    reservation_id=chi.lease.get_node_reservation(head_lease['id']),
    image_name=head_image,
    network_name=net_name,
    key_name=ssh_key
)

compute = chi.server.create_server(
    "CD_Compute",
    reservation_id=chi.lease.get_node_reservation(compute_lease['id']),
    image_name=compute_image,
    network_name=net_name,
    key_name=ssh_key,
    count=2
)

### Run nodes

We now actually start the nodes and allocate a floating IP to the head node so we can SSH in.

In [ ]:
import chi.server

head_start = chi.server.wait_for_active(head.id)
compute_start = chi.server.wait_for_active(compute.id)
chi.server.associate_floating_ip(head.id)
print("Instances should now be running")

## Running iperf3

Unfortunately the iperf3 test between the compute nodes has to be done manually. Once you have successfully connected to the head node:
- copy your private key to the .ssh/id_rsa file (remember to chmod 0600 on it)
- ssh into one of the compute nodes from the head node, and run `iperf3 -s`
- ssh into the other compute node, and run `iperf3 -c FIRST_COMP_NODE_IP -J > results.json
- you can then copy this results file

## Plotting

The following code should plot the results, if you are trying to reproduce the result, replace the contents of results_closed_network.json with your results from the last step.

In [ ]:
from matplotlib import pyplot as plt
import json


def read_data(file):
    data = []
    with open(file) as rfile:
        rjson = rfile.read()
        rdata = json.loads(rjson)
        intervals = rdata["intervals"]
        for interval in intervals:
            data.append(interval["sum"]["bits_per_second"])
    return data


def plot(data):
    plt.plot(data)
    plt.title("iperf3 bandwith test on shared network")
    plt.xlabel("interval number")
    plt.ylabel("bits per second")
    plt.show()


def main():
    data = read_data("results_closed_network.json")
    plot(data)


if __name__ == "__main__":
    main()